# ITA_weather project web scraping

### Goal of this part is to download  monthly data-tables of daily weather data of certain Italian cities (eg. see https://www.ilmeteo.it/portale/archivio-meteo/Roma/2019/Agosto). These will be turned into panda dataframes and exported as csv-files. 
### Specifically, we want to find all monthly tables in the period 1985-2019 (included) for every city which has available all of the monthly tables during this period. Eg. if a city x is missing the table for July 2005 it will be excluded altogether. In the end we will obtain 21,840 tables from 52 cities.

In [1]:
import requests
from bs4 import BeautifulSoup as Bsoup

# Part 1
## ITA_weather_link_finder object construction

## First we define a new class - ITA_weather_link_finder - which will find links to webpages with the tables. First, it will find links to all such tables on the  ilmeteo.it web-portal. Then it will filter away cities for which there are missing tables during the period. It will also divide links into a given number of groups - for a bigger convenience of later scraping.

## The class offers great flexibility. First, we can input start and end year of the period we want to scrape. We also input number of partitions into which we want to divide the final links. The default values are the ones we use for our project - 1985, 2019 and 2, respectively.
## Next, the method "region_pages" provides regions' names ("region_names" attribute) and links from the initial "https://www.ilmeteo.it/portale/archivio-meteo/" page to the webpages of Italian regions ("reg_links" attribute, which is also the output of the method). 
## Method "city_pages" uses regional webpages to find list of cities ("cities" attribute) of all regions and links to the cities' data webpages ("cities_links" attribute, which is also the output of the method). User can limit the search to specifically chosen regions by manipulating "reg_links" attribute obtained by the "region_pages" method.
## Method "monthly_pages" requires a list of cities as the input, which is saved as "cities_subset" attribute. This way the search for the tables can be limited to a chosen set of cities. Inputing "cities" attribute will result in the search for all of the cities obtained by the previous method. The links corresponding to the chosen citites are then used to obtain links to the pages with the final data-tables ("month_links" attribute, which is also the output of the method). Given that all cities from all of the regions are inputed, the resulting "month_links" attribute gives, to the best of our knowledge,  links to all of the monthly data-tables available on the web-portal from the period given by the start and end years. User is therefore not limited to use only links later filtered in the way we have chosen. "monthly_pages" method also provides attributes "years_per_city", which shows for what years there is at least 1 table available for the corresponding city in the inputed list of cities, and "months_per_city", which is a list of libraries in which to each year from the previous attribute months with table available  are listed. 
## Method "filtering_cities" returns attribute "filtered_cities", which is a list of those cities in "cities_subset" attribute (ie. cities chosen as the input of the "monthly_pages" method) that have tables available for all months in the researched period.
## Method "partitioning" returns "partitioned_links", which are links to data-tables for filtered cities divided into the given number of partitions (p). It is list of p-elements, each element is list of links.
## Finally, method "partitioned_date_city" construct attributes "partitioned_cities", "partitioned_years" and "partitioned_months", which are lists of lists of cities, years and months corresponding to each link in "partitioned_links" attribute. It also orders link alphabetically by cities, then by years and months from the oldest to the most recent ones.
 
### Creation of an object of ITA_weather_link_finder class requires importation of "requests" and "BeautifulSoup".

In [2]:
class ITA_weather_link_finder:
    
    '''  This weather link finder searches ilmeteo.it website. It creates links to monthly tables with daily weather data in
         Italian cities. It filters through only cities for which data for each month in the chosen interval 
         are available at the website.  
         
         ITA_weather_link_finder class requires importation of requests and BeautifulSoup. '''
    def __init__(self, start_year=1985, end_year=2019, partitions=2):
        
        ''' At the initialization of the object we can choose the start and end years. Links to data from January to December 
        of each year between the start and end year (included) will be then found. The default values which we
        use in our project are 1985 and 2019, respectively.
        
        In addition, option "partitions" decides into how many parts are the final links partitioned (with respect
        to the chosen cities). This will enable user to download data by parts in order to be more time efficient 
        or if they are satisfied with only incomplete data.   '''
            
        self.start_year=start_year
        self.end_year=end_year
        self.partitions=partitions
        self.basic_link="https://www.ilmeteo.it/portale/"
        
        a=self.start_year
        chosen_years=[a]
        while a<self.end_year:                # getting the list of chosen years
            chosen_years.append(a+1)
            a+=1 
        self.chosen_years=chosen_years
        
    def __repr__(self):
        return f"Italian weather data link finder for interval {self.start_year} - {self.end_year}."
       
    
    def region_pages(self):
        '''   This method finds links to the pages of the regions and their names.    '''
        self.link2="https://www.ilmeteo.it/portale/archivio-meteo/"
        beginning = requests.get(self.link2) 
        soup_beginning=Bsoup(beginning.text)
        horalka=soup_beginning.findAll("td")
        rumba=horalka[1].findAll("a")
        reg_partial_links=[x.get("href") for x in rumba]
        self.reg_links=[self.basic_link+region for region in reg_partial_links]   # complete links
        region_names=[x.text for x in rumba]                       # list of all regions
        self.region_names=region_names
        return self.reg_links
    
    def city_pages(self):
        '''  This method finds links to the data pages for the cities and their names.
        List of the regions to which search should be limited can be modified by changing "reg_links" attribute.
        '''
        cities=[]
        cities_partial_links=[]
        for reg_link in self.reg_links:
            reg_page = requests.get(reg_link)
            soup_reg_page=Bsoup(reg_page.text)
            tea=soup_reg_page.findAll("div",{"class":"block noborder"})
            cofee=tea[0].findAll("a",{"target":""})
            for x in cofee:
                cities_partial_links.append(x.get("href"))   # links
                cities.append(x.text)                  # city names
        self.cities=sorted(list(set(cities)))    # to get rid of the reccurent cities, setting alphabetical order
        cities_partial_links=sorted(list(set(cities_partial_links)))   # to get rid of the reccurent cities, setting alphabetical order
        self.cities_links=[self.basic_link+city_link for city_link in cities_partial_links]
        return self.cities_links
    
    def monthly_pages(self, cities_list):
        '''  This method gets links to the monthly data from the pages of the cities. It requires a list of cities
        as input. It enables user to choose only particular cities for link searching. Inputing original "cities" attribute 
        as cities_list will make use of all of the cities available on the website (respectively limited to the regions 
        chosen in the previous step).  '''
        self.cities_subset=cities_list
        if cities_list==self.cities:
            chosen_cities_links=self.cities_links
        else:
            indeces=[self.cities.index(x) for x in cities_list]
            chosen_cities_links=[self.cities_links[x] for x in indeces]
        city_years=[]
        city_months=[]
        MONTH_links=[]
        for city in chosen_cities_links:
            YEARS=[]
            MONTHS=[]
            page = requests.get(city)
            soup1= Bsoup(page.text)
            temps_dark = soup1.findAll("tr",{'class':'dark'})  # links are inside a table with altering dark and light rows
            temps_light = soup1.findAll("tr",{'class':'light'})
            for x in temps_dark:
                tds=x.findAll("td")
                YEARS.append(tds[0].text)    # finds the year on the line
                links=[y.get("href") for y in tds[1].findAll("a")]   # gets links to all months on the line
                months=[y.text for y in tds[1].findAll("a")]    # gets name of all months on the line
                MONTH_links.append(links)  
                MONTHS.append({tds[0].text:months})      # building up the library of years:corresponding_months
            for x in temps_light:                   # repeat the previous for the light rows
                tds=x.findAll("td")
                YEARS.append(tds[0].text)
                links=[y.get("href") for y in tds[1].findAll("a")]
                months=[y.text for y in tds[1].findAll("a")]
                MONTH_links.append(links)
                MONTHS.append({tds[0].text:months})
            city_years.append(sorted(YEARS))    # list of lists of years for each city during which at least some data were recorded
            city_months.append([y for x,y in sorted([(list(x.keys()),x) for x in MONTHS])])  # list of months from which data are available
        self.years_per_city=city_years  # for each city, what years are available
        self.months_per_city=city_months  # for each city, library of available months corresponding to a year
        
        ###     Making working links to the available monthly data-tables   
        final_links=[]
        for city in MONTH_links:
            for monthly_link in city:
                final_links.append(self.basic_link+monthly_link)
        self.month_links=final_links 
        self.MONTH_links=MONTH_links
        return self.month_links
    
    def  filtering_cities(self):
        ''' This method filters away cities for which there are some missing monthly tables during the required period. '''
        cities=self.cities_subset
        good_cities=[]
        good_months=[]
        good_years=[]
        for i in range(len(cities)):
            if len(self.years_per_city[i])!=0:
                good_cities.append(cities[i])
                good_months.append(self.months_per_city[i])
                good_years.append(self.years_per_city[i])
        chosen_years=self.chosen_years
        chosen_cities=[]
        for city in good_cities:          # getting the list of cities for which all months in the chosen years are provided
            adidas=[]
            for year in chosen_years:
                string=city+"/"+str(year)+"/"+"Gennaio"
                indeces=[self.MONTH_links.index(y) for y in self.MONTH_links for x in y if string in x] #is link for the January of the chosen
                if len(indeces)!=0:                                                              # year in the list?
                    index=indeces[0]
                    if len(self.MONTH_links[index])==12:
                        adidas.append(1)
            if sum(adidas)==len(chosen_years):
                chosen_cities.append(city)
        self.filtered_cities=chosen_cities    # cities for which all required tables are available
        return self.filtered_cities
    def partitioning(self): 
        '''  Here we divide filtered cities into groups as decided by the stated number of partitions 
                 and create the final, partitioned links which we will finally use for scraping.
                 Along the way, it orders our links alphabetically, from January of the starting year
                 to December of the end year. '''
        partitioned_cities=[self.filtered_cities[i::self.partitions] for i in range(self.partitions)]
        partitioned_links=[]
        for x in partitioned_cities:
            partitioned_links1=[]
            for link in self.month_links:
                for city in x:
                    for year in self.chosen_years:
                        if city in link and str(year) in link:
                            partitioned_links1.append(link)
            partitioned_links1=sorted(partitioned_links1)
            month_indeces=[4,3,8,1,7,5,6,0,11,10,9,2]
            for r in range(int(len(partitioned_links1)/12)):  # ordering months from January to December
                partitioned_links1[(r*12):((r+1)*12)] =[partitioned_links1[(r*12):((r+1)*12)][i] for i in month_indeces]
            partitioned_links.append(partitioned_links1)
        self.partitioned_links=partitioned_links
        return self.partitioned_links
    def partitioned_date_city(self):
        ''' Making lists of cities, years and months corresponding to the lists of the partitioned links.  '''
        partitioned_cities=[]
        partitioned_months=[]
        partitioned_years=[]
        for x in self.partitioned_links:
            partitioned_cities.append([y.split("/")[5] for y in x])
            partitioned_years.append([y.split("/")[6] for y in x])
            partitioned_months.append([y.split("/")[7] for y in x])
        self.partitioned_cities=partitioned_cities
        self.partitioned_years=partitioned_years
        self.partitioned_months=partitioned_months
          

In [3]:
''' We initialize our link finder with the default options. '''
download=ITA_weather_link_finder()

''' We run the first method - region_pages - in order to get the links to all region pages and their names. '''
reg_pages=download.region_pages()
reg_names=download.region_names

In [4]:
''' Next, we get links and names of the cities from the chosen regions. We use all of the available regions that we got 
      in the previous step. '''
city_pages=download.city_pages()
cities=download.cities

In [5]:
''' Now we acquire links to pages with the monthly data tables. These data tables are what we are looking for.
    However, we are interested only in cities for which there are no missing monthly tables throughout the whole period.
    Thus we also acquire lists of available years and corresponding months for each city. '''
monthly_pages=download.monthly_pages(cities)
avail_years=download.years_per_city   
avail_months=download.months_per_city

In [6]:
len(monthly_pages)

40730

## Altogether we have 40,730 available links. We will subset links for the cities which have available data for each month during the 1985-2019 period.

### Note: for some reason some of the links lead to a no-data page (links are correct, we get the same result if we move directly on the Italian webpage)

In [7]:
''' Once we have information about data availability we can filter those cities which we do not want.  '''
filtered_cities=download.filtering_cities()
years_chosen=download.chosen_years

In [8]:
len(filtered_cities)

52

## We get 52 cities. For 35 years we would have 21,840 monthly data tables.

## Here we decide into how many groups we split our links into. For the default value of 2, the download of 1 group takes approximately 2 hours (for my computer). Take this into consideration while trying to run the code.

In [9]:
''' We now divide the final links into groups as decided by the partitions argument. '''
#download.partitions=4
parted_links=download.partitioning()

In [10]:
len(parted_links[0])

10920

## So for 2 partitions we get 10,920 links in both groups to each month in the 1985-2019 period for 26 Italian cities.

In [11]:
''' Finally we assign to each link corresponding city, year and month. We will use this during downloading for creation of
     data tables. '''
download.partitioned_date_city()
parted_cities=download.partitioned_cities
parted_years= download.partitioned_years
parted_months= download.partitioned_months

# PART 2
## Data scraping

In [12]:
import timeit
from datetime import datetime
import pandas as pd

## An object of class ITA_weather_link_finder will now be used for downloading the monthly tables of daily data from the ilmeteo.it website. This will be performed with the "downloader" function. This function uses results of partitioning and  partitioned_date_city method of a ITA_weather_link_finder object. Single use of this function performs downloads for the chosen partition of links provided by the inputed object. 

## Downloader's inputs are the object, the index number of the links' partition that we want to use, and optional lists of failed links and their indeces. The ouput is a three-item list of:
* ## list of lists of dictionaries ready to be turned to panda dataframe tables; in case of a failed download the corresponding item in the output list is ["error"]
* ## list of links for which download of the table failed
* ## list of indeces corresponding to the above mentioned links

## Default input for lists of failed links and their indeces are empty lists but they can be also  pre-existing lists to which new items will be added at their ends.

In [13]:
def downloader(link_finder,ind, list_of_f_links=[],list_of_f_indeces=[]):    #link_finder is some object of ITA_weather_link_finder class;  
    '''     This function utilizes an ITA_weather_link_finder object (particularly results of "partitioning"
           and "partitioned_date_city" methods) to download data-tables using the provided webpage links.
        Its output are:
           - a list of libraries with variable names and their values, including name of city, year
           and month to which they correspond 
           - a list of links for which download failed
           - indeces of the failed links
           
        When an error is encountered it shows the index of the link for which the download was unsuccessful. Furthermore
        it adds the failed link into the list of failed links and its index into the corresponding list of indeces. I adds
        a list with item "error" inside of it to the list of weather data.
           
        Each 200 links it also shows the proportion of links already scraped. '''
    j=int(ind)
    table=[]                       # i is index of the junk of partitioned links we want to download 
    links=link_finder.partitioned_links[j]
    for link in links:
        try:
            index=int(links.index(link))
            month=link_finder.partitioned_months[j][index]    # what month do we scrape with the particular link?
            year=str(link_finder.partitioned_years[j][index])      # what year do we scrape?
            city=link_finder.partitioned_cities[j][index]       # what city do we scrape?
            Dark_values=[]
            Light_values=[]
            values=[]
            page = requests.get(link)             # getting linked webpage
            soup1= Bsoup(page.text)
            var=soup1.findAll("table")[3].findAll("th")    # find the list of variable names on the webpage
            variables=["city","year","month"]                    # making the list of variable names...
            [variables.append(x.text) for x in var]     #  ... (for each link separately to check if they are always the same)
            temps_dark = soup1.findAll("tr",{'class':'dark'})    # data are in a table with alternating dark 
            temps_light = soup1.findAll("tr",{'class':'light'})   # and light rows
            for x in temps_dark:
                tds=x.findAll("td")
                dark_value=[city,year,month]
                [dark_value.append(y.text) for y in tds]            # getting list of values from each dark row
                Dark_values.append(dark_value)
            for x in temps_light:
                tds=x.findAll("td")
                light_value=[city,year,month]
                [light_value.append(y.text) for y in tds]          # getting list of values from each light row
                Light_values.append(light_value)
            if len(temps_dark)==(len(temps_light)+1) or len(temps_dark)==len(temps_light):
                for i in range(len(temps_light)):                              
                    values.append(Dark_values[i])                   # list of values for months with even number of days
                    values.append(Light_values[i])                         
            else:
                for i in range(len(temps_light)):
                    values.append(Dark_values[i])                   # list of values for months with odd number of days
                    values.append(Light_values[i])
                values.append(Dark_values[len(temps_light)])
            tab={variables[i]:[x[i] for x in values] for i in range(len(variables))} # creating a monthly dictionary - variable:list_of_values
            table.append(tab)               # in the end, we get a list of monthly dictionaries
            if index%200==0:     # checking the progress
                progress=index/len(links)
                print(str(progress*100)+"%")
                now = datetime.now()
                current_time = now.strftime("%H:%M:%S")
                print(current_time)        
        except:
            table.append(["error"])
            index=links.index(link)
            list_of_f_indeces.append(index)
            list_of_f_links.append(link)
            print(index)     # if errors, where?
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print(current_time)  
    return [table,list_of_f_links,list_of_f_indeces]

In [ ]:
# consider an example with only subset of 100 links
cp=ITA_weather_link_finder(1985,2019)

cp.partitioned_links=[x[0:100] for x in download.partitioned_links]

cp.partitioned_date_city()

len(cp.partitioned_cities[0])

In [ ]:
%%timeit
experiment=downloader(cp,0) # Estimate the time required for the download using timeit

# Download 

In [14]:
''' Download of the first partition. '''
down0=downloader(download,0)

0.0%
19:01:51
1.8315018315018317%
19:04:12
3.6630036630036633%
19:06:23
5.4945054945054945%
19:08:43
7.326007326007327%
19:11:01
9.157509157509157%
19:13:11
10.989010989010989%
19:15:29
12.82051282051282%
19:17:59
14.652014652014653%
19:20:40
16.483516483516482%
19:23:04
18.315018315018314%
19:25:39
20.146520146520146%
19:28:00
21.978021978021978%
19:30:29
23.809523809523807%
19:32:52
25.64102564102564%
19:35:14
27.472527472527474%
19:37:36
29.304029304029307%
19:39:54
31.135531135531135%
19:42:10
32.967032967032964%
19:44:35
34.798534798534796%
19:47:05
36.63003663003663%
19:49:37
38.46153846153847%
19:52:01
40.29304029304029%
19:54:28
42.124542124542124%
19:56:52
43.956043956043956%
19:59:30
45.78754578754579%
20:02:01
47.61904761904761%
20:04:26
49.45054945054945%
20:06:50
51.28205128205128%
20:09:16
53.11355311355312%
20:11:44
54.94505494505495%
20:14:14
56.776556776556774%
20:16:42
58.60805860805861%
20:19:32
60.43956043956044%
20:22:06
62.27106227106227%
20:24:25
64.1025641025641

In [15]:
''' Assigning tables, failed links and their indeces from the download of the first partition. '''
failed_links0=down0[1]
failed_indeces0=down0[2]

tables0=down0[0]
print(len(tables0))
print(len(failed_links0))

10920
0


### If some tables failed to be downloaded (perhaps dut to faulty internet connection), we can try to get them again. All we need to do is to declare a new ITA_weather_link_finder object with $partitions=1$, assign the list of failed links as its partitioned_links attribute and run partitioned_date_city method. This object can be then used as the input of the downloader function. 
### Take care that partitioned_links attribute should be a list of lists.

In [ ]:
additions=ITA_weather_link_finder(partitions=1)
additions.partitioned_links=[failed_links0]
additions.partitioned_date_city()
add_down=downloader(additions,0)

In [ ]:
add_tables=add_down[0]
len(add_down[1])==0  # still some errors?

## We can replace ["error"] items in our list of tables with the added tables. In case some tables still failed to download, try to repeat the procedure for the "additions" object itself. We should be able to download all of the tables.

In [ ]:
for i in range(len(failed_indeces0)):
               tables0[failed_indeces0[i]]=add_tables[i]

## To get all of the tables, we just repeat the same procedure for other partitions as well.

In [16]:
down1=downloader(download,1)

0.0%
21:14:59
1.8315018315018317%
21:18:10
3.6630036630036633%
21:21:19
5.4945054945054945%
21:24:34
7.326007326007327%
21:28:16
9.157509157509157%
21:32:01
10.989010989010989%
21:34:42
12.82051282051282%
21:37:29
14.652014652014653%
21:39:54
16.483516483516482%
21:42:46
18.315018315018314%
21:45:22
20.146520146520146%
21:47:53
21.978021978021978%
21:50:17
23.809523809523807%
21:52:34
25.64102564102564%
21:54:54
27.472527472527474%
21:57:11
29.304029304029307%
21:59:24
31.135531135531135%
22:01:41
32.967032967032964%
22:04:01
34.798534798534796%
22:06:21
36.63003663003663%
22:08:39
38.46153846153847%
22:11:03
40.29304029304029%
22:14:10
42.124542124542124%
22:16:27
43.956043956043956%
22:18:43
45.78754578754579%
22:20:58
47.61904761904761%
22:23:21
49.45054945054945%
22:25:43
51.28205128205128%
22:28:04
53.11355311355312%
22:30:24
54.94505494505495%
22:32:41
56.776556776556774%
22:34:57
58.60805860805861%
22:37:12
60.43956043956044%
22:39:37
62.27106227106227%
22:41:54
64.1025641025641

In [17]:
''' Assigning tables, failed links and their indeces from the download of the second partition. '''
failed_links1=down1[1]
failed_indeces1=down1[2]

tables1=down1[0]
print(len(tables1))
print(len(failed_links1))

10920
0


## We can now check whether are the names and order of the variables inside of each table the same, ie. if all tables have the same structure. We then transform the tables inside proper panda dataframes and export them as csv files for the use in next parts of the project.

In [18]:
'''  Are the name of the variables always the same (including their order)?    '''
print(sum([list(x.keys())==list(tables0[0].keys()) for x in tables0])==len(tables0))
print(sum([list(x.keys())==list(tables1[0].keys()) for x in tables1])==len(tables1))

True
True


In [19]:
'''    Creating list of dataframes - each month of each city is a single dataframe  '''
list_of_DF0=[pd.DataFrame(x) for x in tables0]

In [20]:
'''    Creating list of dataframes  '''
list_of_DF1=[pd.DataFrame(x) for x in tables1]

In [21]:
'''     Exporting dataframes to the csv files        '''
df_paths0=['d:\moje_dokumenty\Desktop\IES\semester 11\Python\project\DATA1\ '+str(10000+tables0.index(x))+".csv" for x in tables0]
[list_of_DF0[df_paths0.index(x)].to_csv(x, index = None, header=True) for x in df_paths0]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [22]:
'''     Exporting dataframes to the csv files        '''
df_paths1=['d:\moje_dokumenty\Desktop\IES\semester 11\Python\project\DATA02\ '+str(10000+tables1.index(x))+".csv" for x in tables1]
[list_of_DF1[df_paths1.index(x)].to_csv(x, index = None, header=True) for x in df_paths1]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,